In [5]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

/home/khim3/.local/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
df = pd.read_csv('./movies.csv')
df.head()

,plot,runtime,genres,fullplot,directors,writers,countries,poster,languages,cast,title,num_mflix_comments,rated,imdb,awards,type,metacritic,plot_embedding
0,Young Pauline is left a lot of money when her ...,199.0,['Action'],Young Pauline is left a lot of money when her ...,"['Louis J. Gasnier', 'Donald MacKenzie']","['Charles W. Goddard (screenplay)', 'Basil Dic...",['USA'],https://m.media-amazon.com/images/M/MV5BMzgxOD...,['English'],"['Pearl White', 'Crane Wilbur', 'Paul Panzer',...",The Perils of Pauline,0,NaN,"{'id': 4465, 'rating': 7.6, 'votes': 744}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[-0.009285833686590195, -0.005062106531113386,..."
1,A penniless young man tries to save an heiress...,22.0,"['Comedy', 'Short', 'Action']",As a penniless man worries about how he will m...,"['Alfred J. Goulding', 'Hal Roach']",['H.M. Walker (titles)'],['USA'],https://m.media-amazon.com/images/M/MV5BNzE1OW...,['English'],"['Harold Lloyd', 'Mildred Davis', ""'Snub' Poll...",From Hand to Mouth,0,TV-G,"{'id': 10146, 'rating': 7.0, 'votes': 639}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.0024393736384809017, 0.02309591695666313, ..."
2,"Michael ""Beau"" Geste leaves England in disgrac...",101.0,"['Action', 'Adventure', 'Drama']","Michael ""Beau"" Geste leaves England in disgrac...",['Herbert Brenon'],"['Herbert Brenon (adaptation)', 'John Russell ...",['USA'],NaN,['English'],"['Ronald Colman', 'Neil Hamilton', 'Ralph Forb...",Beau Geste,0,NaN,"{'id': 16634, 'rating': 6.9, 'votes': 222}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.012204291298985481, -0.011455751024186611, ..."
3,"Seeking revenge, an athletic young man joins t...",88.0,"['Adventure', 'Action']",A nobleman vows to avenge the death of his fat...,['Albert Parker'],"['Douglas Fairbanks (story)', 'Jack Cunningham...",['USA'],https://m.media-amazon.com/images/M/MV5BMzU0ND...,NaN,"['Billie Dove', 'Tempe Pigott', 'Donald Crisp'...",The Black Pirate,1,NaN,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.004541347734630108, -0.0006100571481510997,..."
4,An irresponsible young millionaire changes his...,58.0,"['Action', 'Comedy', 'Romance']","The Uptown Boy, J. Harold Manners (Lloyd) is a...",['Sam Taylor'],"['Ted Wilde (story)', 'John Grey (story)', 'Cl...",['USA'],https://m.media-amazon.com/images/M/MV5BMTcxMT...,['English'],"['Harold Lloyd', 'Jobyna Ralston', 'Noah Young...",For Heaven's Sake,0,PASSED,"{'id': 16895, 'rating': 7.6, 'votes': 918}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.0022256113588809967, 0.011567804031074047,..."


In [3]:
# get na values of each column
df.isna().sum()

plot                   27
runtime                15
genres                  0
fullplot               48
directors              13
writers                13
countries               0
poster                 89
languages               1
cast                    1
title                   0
num_mflix_comments      0
rated                 308
imdb                    0
awards                  0
type                    0
metacritic            928
plot_embedding          0
dtype: int64

In [4]:
df = df.drop(columns=['plot_embedding'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   plot                1473 non-null   object 
 1   runtime             1485 non-null   float64
 2   genres              1500 non-null   object 
 3   fullplot            1452 non-null   object 
 4   directors           1487 non-null   object 
 5   writers             1487 non-null   object 
 6   countries           1500 non-null   object 
 7   poster              1411 non-null   object 
 8   languages           1499 non-null   object 
 9   cast                1499 non-null   object 
 10  title               1500 non-null   object 
 11  num_mflix_comments  1500 non-null   int64  
 12  rated               1192 non-null   object 
 13  imdb                1500 non-null   object 
 14  awards              1500 non-null   object 
 15  type                1500 non-null   object 
 16  metacr

In [8]:
model = SentenceTransformer('thenlper/gte-large')
def get_embedding(text: str) -> list[float]:
    if not isinstance(text, str) or not text.strip():  # Check if the input is not a string or is empty
        return []  # Return an empty list for non-string values or empty strings
    embedding = model.encode(text)
    return embedding.tolist()

# Apply the function to the 'fullplot' column
df['plot_embedding'] = df['fullplot'].apply(get_embedding)

# Display the first few rows to verify
df.head()

,plot,runtime,genres,fullplot,directors,writers,countries,poster,languages,cast,title,num_mflix_comments,rated,imdb,awards,type,metacritic,plot_embedding
0,Young Pauline is left a lot of money when her ...,199.0,['Action'],Young Pauline is left a lot of money when her ...,"['Louis J. Gasnier', 'Donald MacKenzie']","['Charles W. Goddard (screenplay)', 'Basil Dic...",['USA'],https://m.media-amazon.com/images/M/MV5BMzgxOD...,['English'],"['Pearl White', 'Crane Wilbur', 'Paul Panzer',...",The Perils of Pauline,0,NaN,"{'id': 4465, 'rating': 7.6, 'votes': 744}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[-0.009285841137170792, -0.00506209721788764, ..."
1,A penniless young man tries to save an heiress...,22.0,"['Comedy', 'Short', 'Action']",As a penniless man worries about how he will m...,"['Alfred J. Goulding', 'Hal Roach']",['H.M. Walker (titles)'],['USA'],https://m.media-amazon.com/images/M/MV5BNzE1OW...,['English'],"['Harold Lloyd', 'Mildred Davis', ""'Snub' Poll...",From Hand to Mouth,0,TV-G,"{'id': 10146, 'rating': 7.0, 'votes': 639}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.002439397620037198, 0.023095915094017982, ..."
2,"Michael ""Beau"" Geste leaves England in disgrac...",101.0,"['Action', 'Adventure', 'Drama']","Michael ""Beau"" Geste leaves England in disgrac...",['Herbert Brenon'],"['Herbert Brenon (adaptation)', 'John Russell ...",['USA'],NaN,['English'],"['Ronald Colman', 'Neil Hamilton', 'Ralph Forb...",Beau Geste,0,NaN,"{'id': 16634, 'rating': 6.9, 'votes': 222}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.01220429316163063, -0.011455749161541462, -..."
3,"Seeking revenge, an athletic young man joins t...",88.0,"['Adventure', 'Action']",A nobleman vows to avenge the death of his fat...,['Albert Parker'],"['Douglas Fairbanks (story)', 'Jack Cunningham...",['USA'],https://m.media-amazon.com/images/M/MV5BMzU0ND...,NaN,"['Billie Dove', 'Tempe Pigott', 'Donald Crisp'...",The Black Pirate,1,NaN,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.0045413663610816, -0.0006100514438003302, -..."
4,An irresponsible young millionaire changes his...,58.0,"['Action', 'Comedy', 'Romance']","The Uptown Boy, J. Harold Manners (Lloyd) is a...",['Sam Taylor'],"['Ted Wilde (story)', 'John Grey (story)', 'Cl...",['USA'],https://m.media-amazon.com/images/M/MV5BMTcxMT...,['English'],"['Harold Lloyd', 'Jobyna Ralston', 'Noah Young...",For Heaven's Sake,0,PASSED,"{'id': 16895, 'rating': 7.6, 'votes': 918}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.0022255999501794577, 0.011567797511816025,..."


In [9]:
# save df to csv
df.to_csv('movies_embedded.csv', index=False)